In [ ]:
import polars as pl
import openai
from babydragon.memory.indexes.numpy_index import NpIndex
from babydragon.models.embedders.ada2 import OpenAiEmbedder

In [ ]:
# Initialize an OpenAiEmbedder instance
openai_embedder = OpenAiEmbedder()
openai.api_key = "sk-"
# Initialize a NpIndex instance using OpenAiEmbedder
values = ['Hello, world!', 'This is a test sentence.', 'OpenAI is amazing!','cake','pie','ice cream', 'Buffer Errors']
embeddings = openai_embedder.embed(values)


In [ ]:
vindex = NpIndex(values, embeddings)

In [ ]:
df = pl.DataFrame({'values': values, 'embeddings': embeddings}, schema={'values': str, 'embeddings':pl.List( inner=pl.Float64) }) #:pl.Array(width=1536, inner=pl.Float64)

In [ ]:
df.head()

In [ ]:
query_text = "Dessert recommendation"
query_embedding = openai_embedder.embed([query_text])

In [ ]:
len(query_embedding)

In [ ]:
top_k = 3
vindex.search(query_text, top_k=top_k)

In [ ]:
embedding_column_name = 'embeddings'
# Add the query as a new column in the DataFrame
n_rows = len(df)
query_1d_series = pl.Series(query_embedding)
query_df_1d = pl.DataFrame({ 'query': query_embedding},  )
print(query_1d_series, type(query_1d_series))
print(query_df_1d, type(query_df_1d))



In [ ]:
query_1d_series_list = pl.Series([query_embedding])
query_df_1d_list = pl.DataFrame({ 'query': [query_embedding]}, schema={'query': pl.List( inner=pl.Float64) })
query_df_bd_list = pl.DataFrame({ 'query': [query_embedding] * n_rows }, schema={'query': pl.List( inner=pl.Float64) })
print(query_1d_series_list, type(query_1d_series_list))
print(query_df_1d_list, type(query_df_1d_list))
print(query_df_bd_list, type(query_df_bd_list))

In [ ]:
embeddings_series = df["embeddings"]
embeddings_df = df.select("embeddings")
print(embeddings_series.head(), type(df["embeddings"]))
print(embeddings_df.head(), type(embeddings_df))


In [ ]:
#get the first row of the embeddings column from the different datastructures
first_element_series = embeddings_series[0]
first_element_df = embeddings_df[0]
print(first_element_series, type(embeddings_series[0]))
print(first_element_df, type(embeddings_df[0]))


In [ ]:
#dot product of the first row of the embeddings column with the query both in series format
print(type(first_element_series), type(query_1d_series))
print(first_element_series.shape, query_1d_series.shape)
first_element_series.dot(query_1d_series)

In [ ]:
first_element_df.head()

In [ ]:
#dot product of the first row of the embeddings column with the query both in df format
print(type(first_element_df), type(query_df_1d))
print(first_element_df.shape, query_df_1d.shape)
element = first_element_df["embeddings"].list.eval(pl.element().explode())
print("Printing element:")
print(type(element),element.shape)

print(element)
normal_multiplcation = first_element_df["embeddings"].list.eval(pl.element()*2)
print(normal_multiplcation, type(normal_multiplcation))
dot_product =first_element_df["embeddings"].list.eval(pl.element())[0].dot(query_1d_series)
print(dot_product, type(dot_product))

dot_product_ineval =first_element_df["embeddings"].list.eval(pl.element().explode().dot(query_1d_series))

print(dot_product_ineval, type(dot_product_ineval))

In [ ]:
dot_product_frame = df.with_columns(df["embeddings"].list.eval(pl.element().explode().dot(query_1d_series),parallel=True).alias("dot_product"))

In [ ]:
print(type(query_1d_series),query_1d_series.shape)

In [ ]:
print(df["embeddings"].shape)
print(df["embeddings"][0].shape)
df["embeddings"]

In [ ]:
dot_product_frame.head()

In [ ]:
extended_df = df.hstack(query_df)
extended_df.head()

In [ ]:
dir(extended_df[embedding_column_name].arr)

In [ ]:
dir(extended_df[embedding_column_name].list)